#### Step 1: Load Cases are combined and saved as Parquet file (all_cs_loadcases.parquet). Convert it into a dataframe.

In [6]:
# Import relevant libraries
import pandas as pd
import os
from concurrent.futures import ThreadPoolExecutor, as_completed

In [7]:
# Location of all_cs_loadcases
all_cs_loadcases = 'C:\\Users\\rimjhims\\Walter P. Moore and Associates\\S17-24001-00 DB Texas Instruments LFAB2 - CheeseSlab\\Cheese Slab Export\\all_cs_loadcases.parquet'

# Convert the combined Load Case parquet file to a dataframe
all_cs_loadcases = pd.read_parquet(all_cs_loadcases)

In [8]:
# Reduce demicals point to a hundredth
cols_to_reduce = ['P', 'V2', 'V3', 'T', 'M2','M3']

# Reduce decimal points to hundredth place
all_cs_loadcases[cols_to_reduce] = all_cs_loadcases[cols_to_reduce].round(2)

#### Step 2: Extract minimum and maximum force values

In [9]:
# Extract unique Element names
all_elements = all_cs_loadcases['Elm'].unique()
all_elements = all_elements.tolist()

# Extract unique Load Cases
all_loadcases = all_cs_loadcases['LoadCase'].unique()
all_loadcases = all_loadcases.tolist()

In [10]:
# Define a function to process a single (element, load_case) pair
def process_element_loadcase(element, load_case, all_cs_loadcases):

    # Extract element rows
    relevant_rows = all_cs_loadcases[(all_cs_loadcases['Elm'] == element) & (all_cs_loadcases['LoadCase'] == load_case)]

    # If no rows are found, skip the combination
    if relevant_rows.empty:
        return None
    
    # Collect min and max values for each force
    result_row = {
            'Elm': element,
            'LoadCase': load_case,
            'min_P': relevant_rows['P'].min(),
            'max_P': relevant_rows['P'].max(),
            'min_V2': relevant_rows['V2'].min(),
            'max_V2': relevant_rows['V2'].max(),
            'min_V3': relevant_rows['V3'].min(),
            'max_V3': relevant_rows['V3'].max(),
            'min_T': relevant_rows['T'].min(),
            'max_T': relevant_rows['T'].max(),
            'min_M2': relevant_rows['M2'].min(),
            'max_M2': relevant_rows['M2'].max(),
            'min_M3': relevant_rows['M3'].min(),
            'max_M3': relevant_rows['M3'].max(),
        }
    
    return result_row


# Main function to execute threading
def main(all_elements, all_loadcases, all_cs_loadcases):
    result_rows = []

    # Use ThreadPoolExecutor to parallelize the loop
    with ThreadPoolExecutor() as executor:
        futures = []

        # Submit tasks for each element-loadcase pair
        for element in all_elements:
            for load_case in all_loadcases:
                futures.append(executor.submit(process_element_loadcase, element, load_case, all_cs_loadcases))
        
        # Collect results as they complete
        for future in as_completed(futures):
            result = future.result()
            if result:  # Only append non-None results
                result_rows.append(result)
    
    # Create the result DataFrame
    result_df = pd.DataFrame(result_rows)
    return result_df

In [ ]:
result_df = main(all_elements, all_loadcases, all_cs_loadcases)